In [55]:
import re
from datetime import datetime,timedelta
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
#Notice url structure for checkin and checkout date
target_url1 = "https://www.booking.com/hotel/us/gateway-inn-and-conference-center.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaIgCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKZmtG7BsACAdICJGI2YjI5MzQyLTNiM2EtNDRhYi05NWE3LWFkOTZhZjU4NmZkYtgCBeACAQ&sid=f4938e52c9ad18b2aa9ab0bead33fd5d&all_sr_blocks=176350816_107181574_0_34_0&checkin=2025-01-01&checkout=2025-01-02&dest_id=20017561&dest_type=city&dist=0&group_adults=2&group_children=0&hapos=1&highlighted_blocks=176350816_107181574_0_34_0&hpos=1&matching_block_id=176350816_107181574_0_34_0&no_rooms=1&req_adults=2&req_children=0&room1=A%2CA&sb_price_type=total&sr_order=popularity&sr_pri_blocks=176350816_107181574_0_34_0__8900&srepoch=1735675202&srpvid=e19c8c9c98f603fd&type=total&ucfs=1&"

In [96]:
def update_booking_dates(url, checkin_date, checkout_date,):
    """
    Update the checkin and checkout dates in the given booking.com URL.

    Args:
    url (str): The original URL.
    checkin_date (str): The new check-in date in YYYY-MM-DD format.
    checkout_date (str): The new check-out date in YYYY-MM-DD format.

    Returns:
    str: The updated URL with new check-in and check-out dates.
    """
    url = re.sub(r'checkin=\d{4}-\d{2}-\d{2}', f'checkin={checkin_date}', url)
    url = re.sub(r'checkout=\d{4}-\d{2}-\d{2}', f'checkout={checkout_date}', url)
    return url
def is_correct_table(html_table_string):
    #Convert into a string IO object
    str_table = StringIO(str(html_table_string))
    #Read the table into a pandas dataframe
    df = pd.read_html(str_table)[0]
    #Check if the table has the correct columns
    cols = df.columns
    return "Today's price" in cols
def extract_table(target_url,date):
    resp = requests.get(target_url, headers = headers)
    if resp.status_code != 200:
        print(f'Response status: {resp.status_code}')
    soup = BeautifulSoup(resp.text, 'html.parser')
    tables = soup.find_all('table')
    rating = soup.find('div',class_ = 'ac4a7896c7').text.strip().split(' ')[1]
    prop = soup.find('h2',class_ = 'd2fee87262').text.strip()
    
    df = pd.DataFrame()
    for table in tables:
        if is_correct_table(table):
            df = pd.read_html(StringIO(str(table)))[0]
            df["Rating"] = rating
            df['Date'] = date
            df['Competitor'] = prop
            break
    
    if df.empty:
        print("No valid table found")
        return 
    else:
        return df


In [74]:
#Provide the basic url structure for a competitor
url_structure = 'https://www.booking.com/hotel/us/spirit-lake-lodge.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaIgCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKZmtG7BsACAdICJGI2YjI5MzQyLTNiM2EtNDRhYi05NWE3LWFkOTZhZjU4NmZkYtgCBeACAQ&sid=f4938e52c9ad18b2aa9ab0bead33fd5d&all_sr_blocks=40236002_401571595_0_0_0&checkin=2025-01-01&checkout=2025-01-02&dest_id=20017561&dest_type=city&dist=0&group_adults=2&group_children=0&hapos=3&highlighted_blocks=40236002_401571595_0_0_0&hpos=3&matching_block_id=40236002_401571595_0_0_0&no_rooms=1&req_adults=2&req_children=0&room1=A%2CA&sb_price_type=total&sr_order=popularity&sr_pri_blocks=40236002_401571595_0_0_0__12000&srepoch=1735676385&srpvid=e19c8c9c98f603fd&type=total&ucfs=1&'

today = datetime.today()

In [95]:
def find_type_name(df):
    #It's possible to get values like "room type" or "apartment type", this accounts for that variance
    headers = df.columns
    for header in headers:
        if "type" in header.lower():
            return header

def get_room_name(txt):
    #Regex to get the words before the first digit
    pattern = r'^\D*'
    s = re.match(pattern,txt).group().strip()
    if 'Only' in s:
        s = s.partition("Only")[0].strip()
    return s

def clean_df(df):
    if df is None or df.empty:
        return
    #apply functions to clean df
    guests = df["Number of guests"].apply(lambda x: x.split(": ")[1])

    
    pattern = r'(\d+)' #Get the first number from the string
    prices = df["Today's price"].apply(lambda x: re.search(pattern,x).group(0))
    
    type_name = find_type_name(df)
    room_type = df[type_name].apply(get_room_name)
    
    rating = df['Rating']
    date = df['Date']
    competitor = df['Competitor']
    
    new_df = pd.DataFrame([competitor,room_type,guests,prices,rating,date]).T
    new_df.columns = ["Competitor",'Room Type','Guests','Price','Rating','Check-in Date']
    new_df['Report Date'] = datetime.today().strftime('%m-%d-%Y')
    return new_df
def get_data(url_structure,days_out):
    checkin_date = (today + timedelta(days = days_out)).strftime('%Y-%m-%d')
    checkout_date = (today + timedelta(days = days_out + 1)).strftime('%Y-%m-%d')
    new_url = update_booking_dates(url_structure,checkin_date,checkout_date)
    print(f'Gathering data for: {checkin_date}')
    df = extract_table(new_url,checkin_date)
    cleaned = clean_df(df)
    return cleaned
def get_month_out(url_structure):
    df = pd.DataFrame()
    for i in range(1,31):
        df = get_data(url_structure,i) if df.empty else pd.concat([df,get_data(url_structure,i)])
    return df

In [97]:
df = get_month_out(url_structure)

Gathering data for: 2025-01-01
Gathering data for: 2025-01-02
Gathering data for: 2025-01-03
Gathering data for: 2025-01-04
Gathering data for: 2025-01-05
Gathering data for: 2025-01-06
Gathering data for: 2025-01-07
Gathering data for: 2025-01-08
Gathering data for: 2025-01-09
Gathering data for: 2025-01-10
Gathering data for: 2025-01-11
Gathering data for: 2025-01-12
Gathering data for: 2025-01-13
Gathering data for: 2025-01-14
Gathering data for: 2025-01-15
Gathering data for: 2025-01-16
Gathering data for: 2025-01-17
Gathering data for: 2025-01-18
Gathering data for: 2025-01-19
Gathering data for: 2025-01-20
Gathering data for: 2025-01-21
Gathering data for: 2025-01-22
Gathering data for: 2025-01-23
Gathering data for: 2025-01-24
No valid table found
Gathering data for: 2025-01-25
No valid table found
Gathering data for: 2025-01-26
Gathering data for: 2025-01-27
Gathering data for: 2025-01-28
Gathering data for: 2025-01-29
Gathering data for: 2025-01-30


In [99]:
df.to_csv("booking_competitor.csv",index = False,header = True,mode = 'a')